# Chapter 8. Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchial Indexing

**Hierarchical indexing enables you to have multiple(two ot more) index levels on an axis**

In [2]:
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

In [3]:
data

a  1   -0.757125
   2    0.153694
   3    1.308391
b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
d  2   -0.467247
   3    0.499487
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**Hierachical indexing enables concisely select subset of data**

In [5]:
data['b']

1    1.196059
3   -0.802251
dtype: float64

In [6]:
data['b':'c']

b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
dtype: float64

In [7]:
data.loc[['b','d']]

b  1    1.196059
   3   -0.802251
d  2   -0.467247
   3    0.499487
dtype: float64

**Selection from inner level**

In [8]:
data.loc[:,2]

a    0.153694
c    0.100702
d   -0.467247
dtype: float64

unstack **:Rearrange hierarchical data into a DataFrame**

In [9]:
data.unstack()

,1,2,3
a,-0.757125,0.153694,1.308391
b,1.196059,NaN,-0.802251
c,0.230151,0.100702,NaN
d,NaN,-0.467247,0.499487


stack **:Inverse of** unstack

In [10]:
data.unstack().stack()

a  1   -0.757125
   2    0.153694
   3    1.308391
b  1    1.196059
   3   -0.802251
c  1    0.230151
   2    0.100702
d  2   -0.467247
   3    0.499487
dtype: float64

**Either axis in a DataFrame can have a hierarchical index**

In [11]:
frame=pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2],],
                    columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

In [12]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

**Name hierarchical index**

In [13]:
frame.index.names=['key1','key2']

In [14]:
frame.columns.names=['state','color']

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

**Partial column indexing can be used to select groups of columns**

In [16]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

MultiIndex **can be created by itself and reused**

In [20]:
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],
                         names=['state','color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

swaplevel **takes two level numbers or names and returns a new object with the levels interchanges**

In [21]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

sort_index **sorts the data using only the values in a single level**

In [26]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

sort_index **Combine swapping level with sort_index**

In [23]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

**Many descriptive and summary statistics have a** level **option to specify the particular level**

In [27]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's Columns

In [29]:
frame=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],
                   'd':[0,1,2,0,1,2,3]})

In [30]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


set_index **function will create a new DataFrame using one or more of its columns as the index**

In [34]:
frame2=frame.set_index(['c','d'])#by default, columns are removed from the DataFrame
#use drop=False

In [32]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [35]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

pandas.merge **connects rows in DataFrame based on one or more keys**<br>
pandas.concat **concatenates or 'stacks' together objects along an axis**<br>
combine_first **enables splicing together overlapping data to fill in missing values**

### Database-Style DataFrame Joins

***Merge* or *join* operations combin datasets by linking rows using one or more keys**

In [39]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [40]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


**One-to-Many Join**

In [41]:
pd.merge(df1,df2)#merge use the overlapping column names as the keys by default

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [45]:
df3

,1key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [46]:
df4

,rkey,data1
0,a,0
1,b,1
2,d,2


In [47]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')#columns names are different in each object, 
#need to specify them separately

,1key,data1_x,rkey,data1_y
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


**By default,** merge **does an inner join**<br>
**other possible options are** 'left', 'right', 'outer'

In [48]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


**Many-to-many joins form the Cartesian prodcut of the rows**
**the join method only affects the distinct key values**

In [50]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [52]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [60]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [58]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**Merge with multiple keys, pass a list of column names**

In [66]:
left

,key1,key2,lval
0,foo,one,1
1,foo,one,2
2,bar,one,3


In [68]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [69]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4
1,foo,one,1.0,5
2,foo,one,2.0,4
3,foo,one,2.0,5
4,bar,one,3.0,6
5,bar,two,NaN,7


**Overlapping column names**<br>
merge **has a** suffixes **option for specifying strings to append to overlapping names in the left and right DataFrame object**

In [71]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

**Pass** left_index=True **or** right_index=True **to indciate that the indeex should be used as a merge key**

In [13]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [14]:
right1

,group_val
a,3.5
b,7.0


**Default merge method is to intersect the join key, can instead form the union of them with an outer join**

In [16]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


**Merge in hierachical index**

In [23]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [24]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

join **will be an implicitly multiple-key merge if the index is hierarachical**

In [25]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)#join is implictly a multiple-key merge

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


**Default is an inner join, produce an outer join using** how='outer'

In [27]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


**Using indexes of both sides of the merge is possible, pass** left_index=True **and** right_index=True

In [30]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [32]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [33]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


join **DataFrame performs a convenient** join **instance for merging by index(combine together many DataFrame objects having the same similar indexes but non-overlapping columns)**

In [37]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


join **method performs a left join on the join keys by default, exactly preserving the left frame's row index**

In [38]:
left1.join(right1, on='key`')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


**Index on index merge**<br>
**Pass a list of DataFrames to** join **as an alternative to the** concat **function**

In [39]:
another=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                    index=['a','c','e','f'],
                    columns=['New York','Oregon'])

In [40]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [52]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,3.0,4.0,7.0,8.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0


### Concatenanting Along an Axis

**numpy's** concatenate **function**

In [53]:
arr=np.arange(3*4).reshape(3,4)

In [54]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [55]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

concat **function in pandas**
**Series with no overlap**

In [59]:
s1

a    0
b    1
dtype: int64

In [60]:
s2

c    2
d    3
e    4
dtype: int64

In [61]:
s3

f    5
g    6
dtype: int64

**By default,** concat **works along** axis=0

In [62]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

**Passing** axis=1 **will result in a DataFrame**

In [64]:
s4=pd.concat([s1,s2,s3],axis=1)
#result is the sorted union of the indexes

In [65]:
s4

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


**To do intersection, pass** join='inner

In [66]:
s4=pd.concat([s1,s2])

In [67]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,1
c,NaN,2
d,NaN,3
e,NaN,4


In [68]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


**To make the result of concatenation visible**

In [75]:
result=pd.concat([s1,s1,s3],keys=['one','two','three'])

In [76]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [ ]:
result.unstack()

**Combining Series along** axis=1 **, the** keys **become DataFrame column headers**

In [77]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


**Concatenation in DataFrame object**

In [83]:
df1=pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])

In [84]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [88]:
df2=pd.DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])

In [86]:
df2

,three,four
a,5,6
b,7,8


In [87]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    7.0  8.0
c      4   5    NaN  NaN

In [ ]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

**If you pass a dict of objects instead of a list, they dict's keys will be used for the keys option**

In [89]:
pd.concat({'level1':df2,'level2':df2},axis=1)

level1      level2     
   three four  three four
a      5    6      5    6
c      7    8      7    8

**Additional arguments governing how hierarchical index is created**<br>
**1. Name the created** axis **levels with the** names **argument**

In [93]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'],names=['upper','lower'])

upper    level1                                  level2                   
lower         a         b         c         d         b         d        a
0      0.122352  0.740989  0.731826 -1.046573  1.267421 -0.353761  0.01146
1      0.033802 -1.136662  0.654406  0.717974 -0.614353  0.064556 -1.90018
2      0.168946  0.291758 -0.809030  1.828050       NaN       NaN      NaN

In [91]:
df1=pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])

**DataFrame in which the** row **index does not contain any relevant data**

In [94]:
df1

,a,b,c,d
0,0.122352,0.740989,0.731826,-1.046573
1,0.033802,-1.136662,0.654406,0.717974
2,0.168946,0.291758,-0.809030,1.828050


In [92]:
df2=pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])

In [95]:
df2

,b,d,a
0,1.267421,-0.353761,0.01146
1,-0.614353,0.064556,-1.90018


In [96]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,0.122352,0.740989,0.731826,-1.046573
1,0.033802,-1.136662,0.654406,0.717974
2,0.168946,0.291758,-0.809030,1.828050
3,0.011460,1.267421,NaN,-0.353761
4,-1.900180,-0.614353,NaN,0.064556


### Combining Data with Overlap

In [97]:
a=pd.Series([np.nan,2.4,0.0,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])

In [98]:
b=pd.Series([0.,np.nan,2.,np.nan,np.nan,5.],index=['a','b','c','d','e','f'])

In [99]:
a

f    NaN
e    2.4
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [100]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [101]:
np.where(pd.isnull(a),b,a)

array([0. , 2.4, 0. , 3.5, 4.5, 5. ])

combine_first: **performs the equialvalent of** np.where(pd.isnull(a),b,a)

In [102]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.4
f    5.0
dtype: float64

combine_first **in DataFrame: does the same as the Series column by column, "patching" missing data in the calling object with the data from the object you passed**

In [104]:
df1=pd.DataFrame({'a':[1.,np.nan,5.,np.nan],
                 'b':[np.nan,2.,np.nan,6.],
                 'c':range(2,18,4)})

In [105]:
df2=pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                 'b':[np.nan,4.,3.,6.,8.]})

In [106]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [107]:
df2

,a,b
0,5.0,NaN
1,4.0,4.0
2,NaN,3.0
3,3.0,6.0
4,7.0,8.0


In [108]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,3.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

**Reshaping with Hierarchical Indexing**<br>
stack **:"rotates" or pivots the columns in the data to rows**<br>
unstack **:"rotates" or pivots from the columns in the data to the row**

### Using the Stack Method

In [115]:
data=pd.DataFrame(np.arange(6).reshape(2,3),
                  index=pd.Index(['Ohio','Colorado'],name='state'),
                 columns=pd.Index(['one','two','three'],
                 name='number'))

In [116]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


stack **pivots the columns into rows, producing a Series**

In [117]:
result=data.stack()

In [118]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

**From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with** unstack<br>
**By default, the inner most level is unstacked**

In [119]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


unstack **a different level by passing a level number or name**

In [120]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [121]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


unstacking **might introduce missing data if all of the values in the level aren't found in each of the subgroup**

In [122]:
s1=pd.Series([0,1,2,3],index=['a','b','c','d'])

In [ ]:
s2=pd.Series([4,5,6],index=['c','d','e'])

In [123]:
data2=pd.concat([s1,s2],keys=['one','two'])

In [124]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,2.0,3.0,4.0


**Stacking filters out missing data by default**

In [126]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    2.0
     d    3.0
     e    4.0
dtype: float64

In [127]:
data2.unstack().stack(dropna=False)#pass dropna=False to keep the NA values

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    2.0
     d    3.0
     e    4.0
dtype: float64

**When you unstack in a DataFrame,the level unstacked becomes the lowest level in the result**

In [128]:
df=pd.DataFrame({'left':result,'right':result+5},
               columns=pd.Index(['left','right'],
                               name='side'))

In [129]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [130]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

**We can indicate the name of the axis to unstack and unstack**

In [131]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format

**Have a DataFrame containing one column per distinct** item **value indexed by timestamps in the** date **column**<br>
**The first two values passed are the columns to be used respectively as the row and the column index, then finally an optional value column to fill the DataFrame**

In [ ]:
pivoted.ldata('date','item','value')#First k-1 parameters are PKs, only the last one is the value

**To reshape two value columns simulataneously**<br>
**By omitting the last argument, we obtain a DataFrame wih hierarchical columns**

In [ ]:
ldata.pivot('data','item')

pivot **is equivalent to creating a hierarhichal index using** set_index **followed by a call to** unstack

In [ ]:
ldata.set_index(['date','item'].unstack('item'))

### Pivoting "Wide" to "Long" Format

**An inverse operation to** pivot **for DataFrame is** pandas.melt<br>
pandas.melt **merges multiple columns into one,producing a DataFrame that is longer than the input**

In [133]:
df=pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})

In [134]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


**When using** pandas.melt **,we must indicate which columns(if any) are group indicators**

In [135]:
melted=pd.melt(df,['key'])

In [136]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


**Use** pivot **to reshape back to the original layout**

In [137]:
reshaped=melted.pivot('key','variable','value')

In [138]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


**Result of** pivot **creates an index from the columns used as the row lables, can use** reset_index **to move the data back into a column**

In [139]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


**Specify a subset of columns to use as value columns**

In [140]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandas.melt **can be used without any group identifiers**

In [141]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [142]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [143]:
%%bash
git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    .ipynb_checkpoints/Untitled-checkpoint.ipynb
	deleted:    .ipynb_checkpoints/Untitled1-checkpoint.ipynb
	modified:   Data_Loading_Practice.ipynb
	deleted:    Untitled.ipynb
	deleted:    Untitled1.ipynb
	modified:   data_cleaning.ipynb
	modified:   datawrangling.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	.ipynb_checkpoints/Data_Loading_Practice-checkpoint.ipynb
	.ipynb_checkpoints/TimeSeries (1column)-checkpoint.ipynb
	.ipynb_checkpoints/data_cleaning-checkpoint.ipynb
	.ipynb_checkpoints/datawrangling-checkpoint.ipynb
	.ipynb_checkpoints/practice1-checkpoint.ipynb
	Book1.xlsx
	STI (Edited).csv
	TimeSeries (1column).ipynb
	practice1.ipynb
	transaction.csv

no changes added to commit (use "

In [ ]:
git add 